# 신경망 하이퍼파라미터 튜닝하기

케라스 모델을 사이킷런 추정기처럼 동작하도록 하면, 2장에서와 같이 사이킷런의 `GridSearchCV`나 `RandomizedSearchCV`를 통해 가장 좋은 점수를 내는 하이퍼파라미터 조합을 k-fold cross validation을 통해 찾을 수 있다.

캘리포니아 주택 가격 데이터셋을 이용한 회귀 모델을 통해 이를 수행해보자.

In [1]:
# 데이터셋 로드
import os
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

dataset_path = os.path.join(os.getcwd(), "datasets")

if not os.path.isdir(dataset_path):
    os.makedirs(dataset_path)

housing = fetch_california_housing(data_home=dataset_path)

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data,
                                                              housing.target)
X_train, X_val, y_train, y_val = train_test_split(X_train_full,
                                                  y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

먼저, 하이퍼파라미터로 케라스 모델을 만들고 컴파일 하는 함수를 구현한다.

In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers

def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(layers.Dense(n_neurons, activation="relu"))
    model.add(layers.Dense(1))
    model.compile(loss="mse", optimizer=optimizers.SGD(lr=learning_rate))
    return model

`KerasRegressor` 객체는 케라스 모델을 감싸는 간단한 래퍼이다. 이를 사용하면, 사이킷런 회귀 추정기처럼 이 객체를 사용할 수 있다.

`build_model()`에 정의된 기본 하이퍼파라미터를 사용하는 모델을 `KerasRegressor` 객체로 생성한다.

In [3]:
keras_reg = tf.keras.wrappers.scikit_learn.KerasRegressor(build_model)

이제 `fit()`, `score()`, `predict()`를 사용하는 사이킷런처럼 이 객체를 사용할 수 있다.

In [4]:
keras_reg.fit(X_train, y_train, epochs=100,
              validation_data=(X_val, y_val),
              callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 68us/sample - loss: 1.1269 - val_loss: 0.6646
Epoch 2/100
11610/11610 [==============================] - 1s 45us/sample - loss: 0.5889 - val_loss: 0.5385
Epoch 3/100
11610/11610 [==============================] - 1s 45us/sample - loss: 0.5168 - val_loss: 0.4913
Epoch 4/100
11610/11610 [==============================] - 1s 45us/sample - loss: 0.4899 - val_loss: 0.4691
Epoch 5/100
11610/11610 [==============================] - 1s 47us/sample - loss: 0.4694 - val_loss: 0.4576
Epoch 6/100
11610/11610 [==============================] - 1s 46us/sample - loss: 0.4573 - val_loss: 0.4443
Epoch 7/100
11610/11610 [==============================] - 1s 45us/sample - loss: 0.4484 - val_loss: 0.4363
Epoch 8/100
11610/11610 [==============================] - 1s 45us/sample - loss: 0.4413 - val_loss: 0.4300
Epoch 9/100
11610/11610 [==============================] - 1s 45us/sample - loss: 0.431

Epoch 76/100
11610/11610 [==============================] - 0s 43us/sample - loss: 0.3437 - val_loss: 0.3483
Epoch 77/100
11610/11610 [==============================] - 1s 44us/sample - loss: 0.3430 - val_loss: 0.3461
Epoch 78/100
11610/11610 [==============================] - 1s 44us/sample - loss: 0.3439 - val_loss: 0.3463
Epoch 79/100
11610/11610 [==============================] - 1s 44us/sample - loss: 0.3432 - val_loss: 0.3460
Epoch 80/100
11610/11610 [==============================] - 1s 45us/sample - loss: 0.3413 - val_loss: 0.3479
Epoch 81/100
11610/11610 [==============================] - 1s 46us/sample - loss: 0.3424 - val_loss: 0.3441
Epoch 82/100
11610/11610 [==============================] - 1s 44us/sample - loss: 0.3405 - val_loss: 0.3468
Epoch 83/100
11610/11610 [==============================] - 1s 44us/sample - loss: 0.3399 - val_loss: 0.3509
Epoch 84/100
11610/11610 [==============================] - 1s 43us/sample - loss: 0.3415 - val_loss: 0.3481
Epoch 85/100
11610/

In [5]:
# 사이킷런은 손실이 아니라 점수를 계산하므로, score()의 출력은 음수의 MSE가 됨
mse_test = keras_reg.score(X_test, y_test)
print(mse_test)

5160/5160 [==============================] - 0s 26us/sample - loss: 0.3559
-0.35591910227324614


In [6]:
X_new = X_test[:3]
y_pred = keras_reg.predict(X_new)
print(y_pred.ravel(), y_test[:3])

[1.6871035  0.8525628  0.86632776] [2.45  0.661 0.842]


이 문제의 경우, 하이퍼파라미터가 많으므로 그리드 서치가 아닌 랜덤 탐색을 수행하는 것이 좋다.

hidden layer의 수, neuron의 수, learning rate를 사용해 하이퍼파라미터 탐색을 수행해보자.

***(참고) 아래 코드는 실행 시, 에러가 나는데 이는 사이킷런 버전에 따른 버그이다.***

***https://github.com/ageron/handson-ml2/issues/103 에서도 같은 버그를 겪은 사람이 저자의 깃허브에 issue를 하였다.(사이킷런의 버전을 0.21.3으로 downgrade하면 된다는 이야기도 있는것 같은데 나는 넘어가기로 하였다.***

In [ ]:
import numpy as np
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = [{
    "n_hidden" : [0, 1, 2, 3],
    "n_neurons" : np.arange(1, 100),
    "learning_rate" : (reciprocal(3e-4, 3e-2)),
}]

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=1, cv=2, verbose=2)
# RandomizedSearchCV에서는 k-fold cross validation을 사용하므로,
# X_val와 y_val는 사용되지 않으며, 이는 EarlyStopping에만 사용된다.
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_val, y_val),
                  callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

이제 랜덤탐색이 찾은 최상의 하이퍼파라미터로 훈련된 케라스 모델을 가져와보자.

In [ ]:
# 최상의 하이퍼파라미터와 점수
print(rnd_search_cv.best_params_)
print(rnd_search_cv.best_score_)

# 최상의 모델
model = rnd_search_cv.best_estimator_.model

이제 위 모델을 저장하고 test set에서 평가하면 된다.

In [ ]:
model.evaluate(X_test, y_test)

하지만 위의 과정은 매우 오랜 시간이 소요되며, 최상의 방법이 아니다.

더욱 효율적으로 하이퍼파라미터 공간을 탐색하는 방법은 탐색 지역이 좋을 경우, 그 공간에서 더 탐색을 수행하는 것이다.

다음은 하이퍼파라미터 최적화에 사용할 수 있는 몇개의 파이썬 라이브러리들이다.
- Hyperopt
- Hyperas, kopt, Talos
- Keras Tuner
- Scikit-Optimize (skopt)
- Spearmint
- Hyperband
- Sklearn-Deap

## 은닉층 개수

여러 층을 계층적으로 쌓은 심층 신경망은 얕은 신경망보다 파라미터 효율성(parameter efficiency)이 더욱 좋다.
- 얕은 신경망보다 적은 수의 뉴런을 사용해 복잡한 함수를 모델링 할 수 있음
- 따라서, 더 빠른 학습이 가능함

심층 신경망의 hidden layer의 계층별 특징은 다음과 같다.
- 아래쪽의 hidden layer : 저수준의 구조를 모델링(예를 들어, 여러 방향이나 모양의 선 등)
- 중간쪽의 hidden layer : 저수준의 구조를 연결해 중간수준의 구조를 모델링(예를 들어, 사각형이나 원 등)
- 가장 위쪽의 hidden layer : 중간 수준의 구조를 연결해 고수준의 구조를 모델링(예를 들어, 얼굴 등)

요약하자면, 계층적 구조의 DNN은 좋은 솔루션으로 빨리 수렴하는 것에 도움이 되며, 새로운 데이터셋으로의 일반화 성능 향상에도 도움이 된다.

## 은닉 층의 뉴런 개수

적절한 뉴런 수를 설정하는 간단한 방법은 overfitting이 일어나기 전까지 점진적으로 뉴런 수를 늘리는 방법이 있다.

하지만 이보다 더욱 효과적인 방법은 **실제 필요한 것보다 많은 층과 뉴런을 가진 모델을 선택하고, overfitting되지 않도록 early stopping이나 regularization을 사용**하는 것이다.(이를 stretch pants 방식이라고 함)

*참고로 층의 뉴런 수보다 층 수를 늘리는 것이 더욱 이득이 많다고 함*

## 학습률, 배치 크기, 다른 하이퍼파라미터들

### 학습률
낮은 학습률에서 시작해서 점진적으로 높여가며 반복해 모델을 훈련한다. 처음에는 학습률이 커짐에 따라 손실이 줄어들지만, 학습률이 어느 지점보다 커지면 다시 손실이 커질 것이다.

이 경우, 최적의 학습률은 손실이 다시 상승하는 지점보다 조금 아래에 있을 것이다.(일반적으로 상승점보다 약 10배 낮은 지점)

*최적의 학습률은 다른 하이퍼파라미터(특히 배치 크기)에 의존적이므로, 다른 하이퍼파라미터가 수정되면 학습률도 반드시 튜닝해야 한다.*

### 옵티마이저
평범한 미니배치 경사 하강법이 아닌 더 좋은 옵티마이저를 선택하고, 이 옵티마이저의 하이퍼 파라미터를 튜닝할 수 있다.

### 배치 크기
큰 배치는 학습 시간을 크게 단축시킬 수 있지만, 일반화 성능에 영향을 미치므로 작은 배치(2~32)를 사용해야 한다는 논문도 있다.

하지만 반대로 학습률 예열(작은 학습률에서 시작해 점점 높이는 방식)과 같은 다양한 기법을 사용하면 매우 큰 배치크기(8192까지)도 사용할 수 있다는 논문도 있다.

따라서, 학습률 예열을 사용해 큰 배치 크기를 시도해보고, 훈련이 불안정하거나 성능이 좋지 못하다면 작은 배치 크기를 사용하는 것이 좋다.

### 활성화 함수
일반적으로 ReLU가 hidden layer에 사용하기 좋은 기본 활성화 함수이다.

출력층의 활성화 함수는 수행하는 작업에 따라 다르다.

### 반복 횟수
대부분의 경우, 반복횟수는 튜닝할 필요가 없으며 대신 early stopping을 사용하면 된다.